In [1]:
import json
from collections import OrderedDict, Counter
from transformers import GPT2TokenizerFast
from tqdm import tqdm

In [2]:
data = []
with open('data/tldr-training-data.jsonl') as f:
    for line in f:
        data.append(json.loads(line))

In [3]:
def view_sample(idx):
    print(f"content: {data[idx]['content']} \n")
    print(f"summary: {data[idx]['summary']} \n")
    print(f"subreddit: {data[idx]['subreddit']}")
    
view_sample(5)

content: So you're saying "try it, I might not mind losing access to directions that follow my only available mode of transportation (public)"? This isn't a it might be ok but some people don't like it issue like Siri not listening to you  well . This is removing an entire function that I use all the time. It's not worth it and I won't be upgrading. Especially since they still haven't provided a feature that lets you roll anything back to a previous version (which just boggles my mind). If have to roll everything back using a backup of it turned out that, yes, I really can't find my way through the maze of subways, busses, and trains that make up the greater NY area with a combination of 3D flyovers, luck, and  magic . 

summary: you don't seem to understand what's going on here. 

subreddit: apple


In [4]:
len(data)

3084410

In [5]:
catagories = ['relationships',
            'AskReddit',
            'relationship_advice',
            'tifu',
            'dating_advice' ,
            'personalfinance',
            'Advice',
            'legaladvice',
            'offmychest',
            'loseit',
            'jobs',
            'self',
            'BreakUps',
            'askwomenadvice',
            'dogs',
            'running',
            'pettyrevenge',
            'needadvice',
            'travel',
            'Parenting',
            'weddingplanning',
            'Pets',
            'Dogtraining',
            'cats',
            'AskDocs',
            'college',
            'GetMotivated',
            'books',
            'Cooking']

#Whitelist catagories
whitelist = [item for item in data if 'subreddit' in item and item["subreddit"] in catagories]

In [6]:
#Remove duplicates
od = OrderedDict((hash(item['body']), item) for item in whitelist)
whitelist = list(od.values())

In [7]:
#Remove items whose body is longer than 512
#probably could just be done in training

tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

trimmed = []
for item in tqdm(whitelist):
    if len(tokenizer(item['body'])['input_ids']) <= 512:
        trimmed.append(item)


#trimmed = [item for item in whitelist ]

100%|██████████| 846091/846091 [11:56<00:00, 1181.09it/s]


In [8]:
subreddits = [item['subreddit'] for item in trimmed]
keys = list(Counter(subreddits).keys())
vals = list(Counter(subreddits).values())
tot = 0
for key, val in zip(keys, vals):
    tot += val
    print(f'{key}: {val}')
print(f'\ntotal items: {tot}')

AskReddit: 440867
relationship_advice: 27339
relationships: 203277
dating_advice: 8223
pettyrevenge: 2539
dogs: 3039
loseit: 8089
personalfinance: 14348
tifu: 36480
jobs: 4022
Parenting: 3236
BreakUps: 2397
running: 3678
travel: 2330
Dogtraining: 1354
offmychest: 7372
Cooking: 1010
Pets: 1423
GetMotivated: 1706
cats: 1473
legaladvice: 7936
self: 5376
weddingplanning: 2246
books: 3426
Advice: 7413
needadvice: 1848
AskDocs: 1192
college: 1585
askwomenadvice: 2680

total items: 807904


In [9]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(trimmed, test_size = .05)

In [10]:
with open('data/tldr-filtered-test.json', 'w') as outfile:
    json.dump(test, outfile)
    
with open('data/tldr-filtered-train.json', 'w') as outfile:
    json.dump(train, outfile)

In [31]:
#Not important code rn

import os
dirs = os.listdir('comparisons')

#Filter out dirs that do not relate to TL:DR
dirs = [item for item in dirs if 'cnnd' not in item and 'edi' not in item]
dirs = dirs[0:1]
#dirs = dirs[0:1]

comparisons = []
labels = []

for directory in tqdm(dirs):
    data = []
    with open(f'comparisons/{directory}') as f:
        for line in f:
            data.append(json.loads(line))

100%|██████████| 1/1 [00:00<00:00,  6.55it/s]
